In [16]:
a="y"

In [9]:
house="/home/samson/enex.csv"
num="/home/samson/enex.csv".count("/")


In [ ]:
a.

In [11]:
house.split("/")[num]

'enex.csv'

In [13]:
pwd

'/Users/s.eromonsei/my_sandbox'

In [15]:
import os
os.makedirs(os.path.dirname("/Users/s.eromonsei/my_sandbox/Engineering/DataEngineering/airflow/dags/enel-x.py"), exist_ok=True)

In [ ]:
from datetime import datetime, timedelta
from airflow.operators.python_operator import PythonOperator
import pytz
import os
import logging
from airflow.models.dag import DAG
from airflow.utils.dates import days_ago
from airflow.sensors.external_task_sensor import ExternalTaskSensor
from airflow.providers.microsoft.azure.hooks.wasb import WasbHook
from airflow.operators.bash import BashOperator
import pandas as pd



logging.basicConfig(filename='new.log', level=logging.DEBUG,
                    format='(name)s:%(asctime)s:%(levelname)s:%(message)s')


args = {
    "owner": "Samson",
    "start_date": days_ago(1),
    "retries": 3,
    "retry_delay": timedelta(minutes=10),
    "owner": "Samson",
    "email": 


with DAG(
    dag_id="Casio_Battery_Extract_from_Azure_Ingestion_to_awsbucket",
    schedule_interval=timedelta(1),
    tags=["Load to A2 for CAISO Battery Customers"],
    default_args=args,
    catchup=False,
) as dag:

    container_name = "enelx"
    today = datetime.utcnow()
    yesterday = today - timedelta(days=1)
    dateRef = today.strftime('%Y%m%d')
    thresholdTime = pytz.UTC.localize(yesterday)
    localPath="/tmp"
    blobConn=WasbHook(wasb_conn_id="AzureBlob",public_read=False)


    def conn_test():
     """
     Check azure blob connectivity
     """
     try:
   
        return print(blobConn.get_conn())
         
     except:
        return logging.debug("COnnectivity to the blob storage cnuld not be established")
    

    def readblob(container_name,blob_name):
     """
     Check azure blob connectivity
     """
     try:
        if  blobConn.check_for_blob(container_name, blob_name)==True:
            print(True)
            #df = pd.read_csv(f'{blobConn.read_file(container_name, blob_name)}')
            print(blobConn.read_file(container_name, blob_name))
     except:
        logging.error("Blob does not exist in storage")


    def getbloblist():
        """Getting Blob based on the date criteria
        Parameters
        - params container_name (str) – The name of the container
        - Params prefix (str | None) – Filters the results to return only blobs whose names begin with the specified prefix
        """
        
        for blob in blobConn.get_blobs_list(container_name=container_name, prefix="MeterData/uploads/") :
            print(type(blob))
            #readblob(container_name,blob)
            # if   thresholdTime < blob.last_modified :
            #     # write blob to local directory
            #     num = blob.count('/')
            #     blobName = blob.split('/')[num]
            #     print(blobName)
                # dateRef = blob.creation_time.strftime('%Y%m%d')
                # download_file_path = os.path.join(localPath + dateRef, blobName)
                # os.makedirs(os.path.dirname(download_file_path), exist_ok=True)
                
                blobConn.get_file(download_file_path, container_name, blobName)
                        #Download a file from Azure Blob Storage.
                        #file_path (str) – Path to the file to download.
                        #container_name (str) – Name of the container.
                        #blob_name (str) – Name of the blob.
                        # kwargs – Optional keyword arguments that BlobClient.download_blob() takes.

                
        return logging.info("Blobs successful")
  

    Blob_read = PythonOperator(
        task_id="readBlob",
        python_callable=readblob,
        dag=dag
        )

    testblobConn= PythonOperator(
        task_id="conn_test",
        python_callable=conn_test,
        dag=dag
        )

    listBlob= PythonOperator(
        task_id="BlobList",
        python_callable=getbloblist,
        dag=dag
        )

    

    testblobConn >> listBlob